In [42]:
import os
execfile(os.path.join(os.environ["SPARK_HOME"], 'python/pyspark/shell.py'))

import pyspark.sql.functions as f
import pyspark.sql.types as t

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.1.1
      /_/

Using Python version 2.7.12 (default, Nov 19 2016 06:48:10)
SparkSession available as 'spark'.


In [35]:
from pyspark.sql import SparkSession
sparkSession = SparkSession.builder.enableHiveSupport().master("local [2]").getOrCreate()

In [36]:
graphPath = "/data/graphDFSample"

In [ ]:
from pyspark.sql.functions import explode, collect_list, size, col, row_number, count, udf
from pyspark.sql import Window
import itertools

def pairs(a):
    return itertools.product(a,a)

pairs_udf = udf(pairs, t.ArrayType(t.StructType([
    t.StructField("user1", t.IntegerType(), False), 
    t.StructField("user2", t.IntegerType(), False)
])))

reversedGraph = sparkSession.read.parquet(graphPath) \
    .withColumn("friend", explode('friends')) \
    .groupBy("friend") \
    .agg(collect_list("user").alias("users")) \
    .filter(size(col("users")) > 1) \
    .withColumn("user", explode(pairs_udf('users'))) \
    .groupBy(col("user.user1"), col("user.user2")) \
    .agg(count("*").alias("friends_count"))

In [ ]:
reversedGraph.printSchema()

In [ ]:
window = Window.orderBy(col("friends_count").desc())
    
top50 = reversedGraph.withColumn("row_number", row_number().over(window)) \
            .filter(col("row_number") < 50) \
            .select(col("friends_count"), col("user1"), col("user2")) \
            .orderBy(col("friends_count").desc()) \
            .collect()

In [ ]:
for val in top50:
    print '%s %s %s' % val